<a href="https://colab.research.google.com/github/noodlepopllc/LearnVietnamese/blob/main/Colab/Text2Speech2Text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch, gc
import gc
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, \
    T5ForConditionalGeneration, T5Tokenizer, VitsModel, pipeline

In [2]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)

cpu


In [3]:
model = T5ForConditionalGeneration.from_pretrained("NlpHUST/t5-en-vi-small")
tokenizer = T5Tokenizer.from_pretrained("NlpHUST/t5-en-vi-small")
model.to(device)

src = "I hope this model is capable of completely translating this sentence, let us see if it can."
tokenized_text = tokenizer.encode(src, return_tensors="pt").to(device)
model.eval()
summary_ids = model.generate(
                    tokenized_text,
                    max_length=128,
                    num_beams=5,
                    repetition_penalty=2.5,
                    length_penalty=1.0,
                    early_stopping=True
                )
output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(output)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Tôi hy vọng mô hình này có thể dịch toàn bộ câu này, để xem nó có thể không.


In [4]:
model.cpu()
del model
del tokenizer
gc.collect()
torch.cuda.empty_cache()

In [5]:

model = T5ForConditionalGeneration.from_pretrained("NlpHUST/t5-vi-en-small")
tokenizer = T5Tokenizer.from_pretrained("NlpHUST/t5-vi-en-small")
model.to(device)

tokenized_text = tokenizer.encode(output, return_tensors="pt").to(device)
model.eval()
summary_ids = model.generate(
                    tokenized_text,
                    max_length=256,
                    num_beams=5,
                    repetition_penalty=2.5,
                    length_penalty=1.0,
                    early_stopping=True
                )
output2 = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(output2)


I hope this model can translate the whole thing, see if it couldn't.


In [6]:
model.cpu()
del model
del tokenizer
gc.collect()
torch.cuda.empty_cache()

In [7]:
model_name = "VietAI/envit5-translation"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
inputs = [f"en: {src}"]
outputs = model.generate(tokenizer(inputs, return_tensors="pt", padding=True).input_ids.to(device), max_length=512)
output = [output for output in tokenizer.batch_decode(outputs, skip_special_tokens=True)][0].split('vi: ')[1]
print(output)



Tôi hy vọng mô hình này có khả năng dịch hoàn toàn câu này, hãy xem nó có thể không.


In [8]:

model_name = "VietAI/envit5-translation"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
outputs = model.generate(tokenizer([f'vi: {output}'], return_tensors="pt", padding=True).input_ids.to(device), max_length=512)
test = [output for output in tokenizer.batch_decode(outputs, skip_special_tokens=True)][0].split('en: ')[1]
print(test)

I hope this model is able to completely translate this sentence, let's see if it can.


In [9]:
model.cpu()
del model
del tokenizer
gc.collect()
torch.cuda.empty_cache()


In [10]:
import scipy
from IPython.display import Audio, display

model = VitsModel.from_pretrained("facebook/mms-tts-vie")
tokenizer = AutoTokenizer.from_pretrained("facebook/mms-tts-vie")
inputs = tokenizer(output, return_tensors="pt")
with torch.no_grad():
    out = model(**inputs).waveform
    sample = out.numpy()[0]
    rate = model.config.sampling_rate
    scipy.io.wavfile.write("out.wav", rate=rate, data=sample)
    audio = Audio(data=sample, rate = rate, autoplay=False)
    display(audio)

In [11]:
model.cpu()
del model
del tokenizer
gc.collect()
torch.cuda.empty_cache()

In [12]:
transcriber = pipeline("automatic-speech-recognition", model="vinai/PhoWhisper-small")
output = transcriber("out.wav")['text']
print(output)


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

special_tokens_map.json: 0.00B [00:00, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

preprocessor_config.json:   0%|          | 0.00/339 [00:00<?, ?B/s]

Device set to use cpu
`return_token_timestamps` is deprecated for WhisperFeatureExtractor and will be removed in Transformers v5. Use `return_attention_mask` instead, as the number of frames can be inferred from it.
Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
Transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English. This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`. See https://github.com/huggingface/transformers/pull/28687 for more details.


tôi hy vọng mô hình này có khả năng dịch hoàn toàn câu này hãy xem nó có thể không.


In [13]:
del transcriber
gc.collect()
torch.cuda.empty_cache()

In [14]:
model_name = "VietAI/envit5-translation"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
inputs = [f"vi: {output}"]
print(inputs)
outputs = model.generate(tokenizer(inputs, return_tensors="pt", padding=True).input_ids.to(device), max_length=512)
output = [output for output in tokenizer.batch_decode(outputs, skip_special_tokens=True)][0].split('en: ')[1]
print(output)

['vi: tôi hy vọng mô hình này có khả năng dịch hoàn toàn câu này hãy xem nó có thể không.']
I hope this model is able to completely translate this sentence. Let's see if it can.
